In [ ]:
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource, HoverTool, Label, Range1d
from bokeh.plotting import figure, output_notebook, show

from nena_api import get_series_data, obtain_access_token

# Visualization of Danish wind power production (2021)

In this notebook we will display data and statistics on the Danish wind power production last year (2021).The tutorial uses a set of function wrappers around the API requests to make it easier to use. These are located in nena_api.py

In this tutorial the plotting is done using the Bokeh plotting library. Make sure you have
installed this along with the JupyterLab Bokeh extension.

In [ ]:
USER = ""  # Insert your Nena username
PASS = ""  # Insert your Nena password
ROOT = "https://api.nena.no"

In [ ]:
DK_WIND_CAP = 7000  # Wind power capcity in Denmark (MW) in 2021 - data from https://ens.dk/service/statistik-data-noegletal-og-kort/data-oversigt-over-energisektoren.
SERIES_ID = "nhdkwind"

In [ ]:
def get_wind_data(series_id, from_date, to_date):
    """
    Helper function for getting wind data from the API.
    """
    data = get_series_data(
        series_id=SERIES_ID,
        user_auth=obtain_access_token(username=USER, password=PASS, root_url=ROOT),
        from_date=from_date,
        to_date=to_date,
        resolution="hr",
    )

    df = pd.DataFrame.from_dict(
        data["Values"], orient="index", columns=[data["Meta"]["unit"]]
    )
    df.index.name = "Time"
    df = df.sort_values(by="MWh")
    df.reset_index(inplace=True)
    df.Time = df.Time.apply(lambda x: pd.to_datetime(x))

    return df


def calc_stats(df):
    """
    Function for calculating and returning a dictionary of dataframes with each key being a 
    percentile.
    """
    percentiles = [0.25, 0.5, 0.75, 0.9]
    dfs = {}
    for p in percentiles:
        y = df["Rel. Power"].quantile(p)
        xs = np.arange(0, df.shape[0], 1)
        ys = np.ones(df.shape[0]) * y
        arr = np.vstack((xs, ys)).T
        dfs[p] = pd.DataFrame(arr, columns=["Hour", "Power"])
    return dfs

We load the data from last year by using the convenience  keywords "boy-1" and "eoy-1" which are the beginning and end of last year's dates

In [ ]:
df = get_wind_data(SERIES_ID, "boy-1", "eoy-1")
MAX_CAPACITY = np.array(
    (np.arange(0, df.shape[0]), np.ones(df.shape[0]) * 100)
).T  # A mock array for creating a vertical line. See below.


df_stats = df.reset_index()
df_stats["Rel. Power"] = (
    df_stats["MWh"] * 100 / DK_WIND_CAP
)  # We calculate the relative power to installed capacity.
stats = calc_stats(df_stats)  # Calculates percentile statistics

In [ ]:
output_notebook()
source = ColumnDataSource(df_stats)

ANNOTATION_FONT_SIZE = "8pt"

p = figure(x_range=(0, 24 * 365), y_range=(0, 110))

p.line(
    x="index",
    y="Rel. Power",
    line_width=1.5,
    line_color="black",
    source=source,
    legend_label="DK Wind. Prod. 2021",
)

p.line(
    x=MAX_CAPACITY[:, 0],
    y=MAX_CAPACITY[:, 1],
    line_width=1.5,
    line_dash="dashed",
    line_color="red",
)

llb_max = Label(
    x=200,
    y=(DK_WIND_CAP + 50) * 100 / DK_WIND_CAP,
    text=f"Installed Capacity (IC): {DK_WIND_CAP} MW",
    text_font_size=ANNOTATION_FONT_SIZE,
    text_font_style="bold",
)
p.add_layout(llb_max)


for pe in stats:
    if pe == 0.25:
        x_pos = 5200
    else:
        x_pos = 200
    p.line(x=stats[pe].Hour, y=stats[pe].Power, line_dash="dotted", line_width=1.5, line_color="grey")
    achieved_power = stats[pe].Power[100]
    llb = Label(
        x=x_pos,
        y=achieved_power + 0.5,
        text=f"{int(pe*100)}th-pctl achieves less than {int(achieved_power)} % of IC.",
        text_font_size=ANNOTATION_FONT_SIZE,
        text_font_style="bold",
    )
    p.add_layout(llb)


p.xaxis.axis_label = "Hour"
p.yaxis.axis_label = "Production over installed capacity (%-MWh/MWh)"
p.title.text = "Hourly danish wind power production (2021)"
p.legend.label_text_font_size = "8pt"


show(p)